In [42]:
import numpy as np # for general handling of numerical data
import pandas as pd # for dataframes
import librosa
import librosa.display as dis
import matplotlib.pyplot as plt
import scipy
import csv
import os
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from scipy import signal 
from sklearn.covariance import EllipticEnvelope 
from window_slider import Slider
np.random.seed(10)


In [25]:
def readQASS(filepath, fs):
    """
    Read QASS data from binary file format.

    Parameters
    ----------
    filepath : STR
        is the full path to sur file.
    fs : sampling frequency of the measurement
 
    Raises
    ------
    ValueError
        DESCRIPTION.

    Returns
    -------
    df:  a pandas dataframe with time and amplitude.
    """
    time_interval = 1/fs
    
    data = np.fromfile(filepath, dtype='int16')
    time = np.arange(0, len(data)*time_interval, time_interval)
    df = pd.DataFrame({'time': time, 'amplitude':data})
    return df

In [26]:
# creates header of csv file with number of section start/end time, cwt and stft 
def create_header_():
    header = 'nr start end'
    for i in range(1, 21):
        header += f' cwt{i}'
    for i in range(1, 11):
        header += f' stft{i}'
    header = header.split()
    return header

In [27]:
# creates csv file with header from create_header_() and a given name.csv
def create_csv(header,name):
    file = open(name, 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    return name

In [28]:
# split .bin file into 32ms sections
# extracte features from each section and write into csv file
def write_data_2_csv_from_bin(file,name):
    i = 0
    sr = 1562500
    measurement_data = readQASS(file, sr)
    amplitude_data = measurement_data['amplitude'].astype(float)
    # cut last part so length of section can be the same length
    data_length = amplitude_data.size - (amplitude_data.size%50000)
    # find the the number of sections for np.array_split function
    n = data_length/((1562500/1000) * 32)
    # splits data into n arrays of size (1562500/1000) * 32)
    sections = np.array_split(amplitude_data[0:data_length], n)
    # start/end time for localizing anomoly
    start = 0
    end = 32
    # extracte cwt and stft from sections
    for section in sections:
        y = sections[i].to_numpy()
        # extract stft matrix from section
        widths = np.arange(1, 21)
        stftmatr = np.abs(librosa.stft(y, n_fft = 19))
        stftmatr_square = np.square(np.square(stftmatr))
        
        # extract cwt matrix from section
        cwtmatr = signal.cwt(y, signal.ricker, widths)
        cwtmatr_square = np.square(np.square(cwtmatr))
        
        # write data into csv file
        to_append = f'{i}'  
        to_append += f' {start} {end}' 
        for c in cwtmatr_square:
            to_append += f' {np.mean(c)}'
        for s in stftmatr_square:
            to_append += f' {np.mean(s)}'    
        file = open(name, 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
        # increase start/end time
        end = end + 32 
        start = start + 32
        i = i+1
        

In [29]:
# implements sliding window on data 
# writes data into csv file
def write_data_2_csv_from_bin_sliding_window(file,name):
    i = 0
    sr = 1562500
    measurement_data = readQASS(file, sr)
    amplitude_data = measurement_data['amplitude'].astype(float)
    y = amplitude_data.to_numpy()
    
    # calcultes window and overlap size of 32ms and 16ms
    size_window = int((1562500/1000) * 32)
    size_overlap = round(((1562500/1000) * 16))
    
    # uses window_slider libary to implement sliding window
    slider = Slider(size_window,size_overlap)
    slider.fit(y)
    sections = []
    while True:
        window_data = slider.slide()
        # do your stuff
        sections.append(window_data)
        if slider.reached_end_of_list(): break
    
    # start/end time for localizing anomoly 
    start = 0
    end = 32 
    
    # extracte cwt and stft from sections
    for sec in sections:
        y = sections[i]
        widths = np.arange(1, 21)
        stftmatr = np.abs(librosa.stft(y, n_fft = 19))
        stftmatr_square = np.square(np.square(stftmatr))
        cwtmatr = signal.cwt(y, signal.ricker, widths)
        cwtmatr_square = np.square(np.square(cwtmatr))
        to_append = f'{i}'  
        
        to_append += f' {start} {end}' 
        for c in cwtmatr_square:
            to_append += f' {np.mean(c)}'
        for s in stftmatr_square:
            to_append += f' {np.mean(s)}'
            
            
        file = open(name, 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
        i = i+1
        
        # increase start/end time
        start = start + 16
        end = end + 16

In [30]:
# loads csv file
def load_csv(path):
    df = pd.read_csv(path)
    compare = df #df zum vergleichen für AUC
    
    data = df.drop(['start', "end", "nr"],axis=1)

    return data ,compare 
    

In [31]:
# preprocesses data
def preprocess_data(data):
    scaler = StandardScaler()
    processed_data = scaler.fit_transform(np.array(data, dtype = float))
    return processed_data

In [32]:
# isolation forest implementation with sklearn
def train_IF_(data):
    rng = np.random.RandomState(10)
    model = IsolationForest(contamination = outliers_fraction, random_state=rng)
    model.fit(data)
    compare['anomalyIF'] = pd.Series(model.predict(data))
    compare['anomalyIF'] = compare['anomalyIF'].map( {1: 0, -1: 1} )
    
    print(compare['anomalyIF'].value_counts())

In [33]:
# elliptic envelope implementation with sklearn
def train_EE(data):
    model = EllipticEnvelope(random_state=1,contamination = outliers_fraction, support_fraction= 1)
    
    model.fit(data)
    compare['anomalyEE'] = pd.Series(model.predict(data))
    compare['anomalyEE'] = compare['anomalyEE'].map( {1: 0, -1: 1} )
    print(compare['anomalyEE'].value_counts())

In [34]:
# k-means implementation with sklearn
def getDistanceByPoint(data, model):
    distance = []
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.append(np.linalg.norm(Xa-Xb))
    return pd.Series(distance, index=data.index)
def train_kmeans(data):
    pca = PCA(n_components=2)
    data = pca.fit_transform(data)
    # standardize these 2 new features
    min_max_scaler = preprocessing.StandardScaler()
    np_scaled = min_max_scaler.fit_transform(data)
    data = pd.DataFrame(np_scaled)
    n_cluster = range(1, 20)
    kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
    scores = [kmeans[i].score(data) for i in range(len(kmeans))]
    #fig, ax = plt.subplots()
    #ax.plot(n_cluster, scores)
    #plt.show()
    compare['cluster'] = kmeans[1].predict(data)
    compare['principal_feature1'] = data[0]
    compare['principal_feature2'] = data[1]
    distance = getDistanceByPoint(data, kmeans[1])
    number_of_outliers = int(outliers_fraction*len(distance))
    threshold = distance.nlargest(number_of_outliers).min()
    # anomaly21 contain the anomaly result of method 2.1 Cluster (0:normal, 1:anomaly) 
    compare['anomalyKmeans'] = (distance >= threshold).astype(int)
    print(compare['anomalyKmeans'].value_counts())

In [35]:
# one class support vector machine implementation with sklearn 
def train_OCSVM_(data): 
    model =  OneClassSVM(nu=0.95 * outliers_fraction, gamma=0.1) #nu=0.95 * outliers_fraction  + 0.05
    model.fit(data)
    compare['anomalyOCSVM'] = pd.Series(model.predict(data))
    compare['anomalyOCSVM'] = compare['anomalyOCSVM'].map( {1: 0, -1: 1} )
    print(compare['anomalyOCSVM'].value_counts())

In [ ]:
# extracte features from .bin file 
# writes into csv file and runs ml algorithms

# outlier fractions 
outliers_fraction = 0.1
header = create_header_()
root = "Acoustic Emission/Parametervariation/" #root of dataset folder
# create_csv(header,name of csv)
name = create_csv(header,"Fe_KleineKörner_vc150_fa7,5_SIG_Raw.csv")
path = root + "Fe_GroßeKörner_vc050_fa02,5_SIG_Raw.bin" 

# write_data_2_csv_from_bin(path,name) for dividing data into 32ms sections
write_data_2_csv_from_bin(path,name)
# write_data_2_csv_from_bin_sliding_window(file,name) for sliding window 
#write_data_2_csv_from_bin_sliding_window(file,name)
data ,compare = load_csv(name)
processed_data = preprocess_data(data)
print("_______IF_________")
train_IF_(processed_data)
print("__________________")
print("______OCSVM_______")
train_OCSVM_(processed_data)
print("__________________")
print("_______EE_________")
train_EE(processed_data)
print("__________________")
print("______K-Means_____")
train_kmeans(processed_data)
print("__________________")

In [ ]:
# extrate features for each file in folder
outliers_fraction = 0.1
path = "Acoustic Emission/Parametervariation/"
for filename in os.listdir(f'{path}'):
    if not filename.startswith('.') and os.path.isfile(os.path.join(path, filename)):
        file = f'{path}{filename}'
        name = filename.split(".")[0]+".csv"
        header = create_header_()
        name = create_csv(header,name)
        write_data_2_csv_from_bin(file,name)
        #write_data_2_csv_from_bin_sliding_window(file,name)
        data ,compare = load_csv(name)
        processed_data = preprocess_data(data)
        print("..................")
        print(filename)
        print("_______IF_________")
        train_IF_(processed_data)
        print("__________________")
        print("______OCSVM_______")
        train_OCSVM_(processed_data)
        print("__________________")
        print("_______EE_________")
        train_EE(processed_data)
        print("__________________")
        print("______K-Means_____")
        train_kmeans(processed_data)
        print("__________________")
        

In [36]:
# load an already created csv file
outliers_fraction = 0.1

name = "Extracted Data/IWT/Parametervariation/Sliding Window/Fe_GroßeKörner_vc150_fa15_SIG_Rawslide_dataset.csv"
data ,compare = load_csv(name)
processed_data = preprocess_data(data)
print("_______IF_________")
train_IF_(processed_data)
print("__________________")
print("______OCSVM_______")
train_OCSVM_(processed_data)
print("__________________")
print("_______EE_________")
train_EE(processed_data)
print("__________________")
print("______K-Means_____")
train_kmeans(processed_data)
print("__________________")


_______IF_________
0    459
1     51
Name: anomalyIF, dtype: int64
__________________
______OCSVM_______
0    460
1     50
Name: anomalyOCSVM, dtype: int64
__________________
_______EE_________


/Users/matthiaspetry/Library/Python/3.7/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


0    459
1     51
Name: anomalyEE, dtype: int64
__________________
______K-Means_____
0    459
1     51
Name: anomalyKmeans, dtype: int64
__________________


In [37]:
# remove extrated features for overview
anomaly = compare.drop(["cwt1","cwt2","cwt3","cwt4","cwt5","cwt6","cwt7","cwt8","cwt9","cwt10","cwt11","cwt12","cwt13","cwt14","cwt15","cwt16","cwt17","cwt18","cwt19","cwt20","stft1","stft2","stft3","stft4","stft5","stft6","stft7","stft8","stft9","stft10","cluster","principal_feature1","principal_feature2"],axis = 1)

In [38]:
# extract specific anomolies 
find_anomaly = anomaly[anomaly.anomalyIF == 1]

In [39]:
# shows list sections where anomolies are fund 
find_anomaly

,nr,start,end,anomalyIF,anomalyOCSVM,anomalyEE,anomalyKmeans
190,190,3040,3072,1,0,0,0
196,196,3136,3168,1,1,0,0
197,197,3152,3184,1,0,0,0
199,199,3184,3216,1,1,0,0
200,200,3200,3232,1,1,0,1
206,206,3296,3328,1,1,0,1
214,214,3424,3456,1,1,0,1
225,225,3600,3632,1,1,1,1
226,226,3616,3648,1,1,0,1
227,227,3632,3664,1,1,1,1
